In [ ]:
!pip install pillow numpy opencv-python matplotlib pinecone rembg onnxruntime onnxruntime-gpu tensorflow google-cloud-storage

In [1]:
import requests
from PIL import Image
import numpy as np
import cv2
import os
from pinecone import Pinecone
import matplotlib.pyplot as plt

In [2]:
from google.cloud import storage
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../red-freedom-426709-a7-6904e9a53b27.json'
storage_client = storage.Client()
bucket_name = 'datalakes-ing3'

In [3]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
         bucket = storage_client.get_bucket(bucket_name)
         blob = bucket.blob(destination_blob_name)
         blob.upload_from_filename(source_file_name)
         print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [4]:
for filename in os.listdir('../data-local-test/0_raw/'):
    img_path = os.path.join('../data-local-test/0_raw/', filename)
    upload_blob(bucket_name, img_path, f'0_raw/{filename}')

File ../data-local-test/0_raw/001_08194468.jpg uploaded to 0_raw/001_08194468.jpg.
File ../data-local-test/0_raw/001_08212dcd.jpg uploaded to 0_raw/001_08212dcd.jpg.
File ../data-local-test/0_raw/001_21a7d5e6.jpg uploaded to 0_raw/001_21a7d5e6.jpg.
File ../data-local-test/0_raw/001_2288a4f6.jpg uploaded to 0_raw/001_2288a4f6.jpg.
File ../data-local-test/0_raw/001_504d320d.jpg uploaded to 0_raw/001_504d320d.jpg.
File ../data-local-test/0_raw/001_5992faf7.jpg uploaded to 0_raw/001_5992faf7.jpg.
File ../data-local-test/0_raw/001_5ef3e95c.jpg uploaded to 0_raw/001_5ef3e95c.jpg.
File ../data-local-test/0_raw/001_986d6c22.jpg uploaded to 0_raw/001_986d6c22.jpg.
File ../data-local-test/0_raw/001_9adc92c2.jpg uploaded to 0_raw/001_9adc92c2.jpg.
File ../data-local-test/0_raw/001_9cd1160a.jpg uploaded to 0_raw/001_9cd1160a.jpg.
File ../data-local-test/0_raw/001_a51bb26a.jpg uploaded to 0_raw/001_a51bb26a.jpg.
File ../data-local-test/0_raw/001_beebcee2.jpg uploaded to 0_raw/001_beebcee2.jpg.
File

In [5]:
from google.cloud import storage
from rembg import new_session, remove
from pathlib import Path
from PIL import Image
import io

session = new_session()
bucket = storage_client.bucket(bucket_name)

input_dir = '0_raw/'
output_dir = '1_staging/'

blobs = storage_client.list_blobs(bucket_name, prefix=input_dir)

for blob in blobs:
    if blob.name.endswith('.jpg'):
        input_content = blob.download_as_bytes()
        
        output_content = remove(input_content, session=session)
        output_image = Image.open(io.BytesIO(output_content))
        
        if output_image.mode == 'RGBA':
            output_image = output_image.convert('RGB')
        
        output_image = output_image.resize((224, 224), Image.LANCZOS)

        output_buffer = io.BytesIO()
        output_image.save(output_buffer, format='JPEG')
        output_buffer.seek(0)

        filename = Path(blob.name).stem + '.out.jpg'
        output_blob_name = f'{output_dir}{filename}'

        output_blob = bucket.blob(output_blob_name)
        output_blob.upload_from_file(output_buffer, content_type='image/jpeg')

        print(f"Processed and saved: gs://{bucket_name}/{output_blob_name}")

E:\Anaconda_folder\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
E:\Anaconda_folder\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
E:\Anaconda_folder\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Processed and saved: gs://datalakes-ing3/1_staging/001_08194468.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_08212dcd.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_21a7d5e6.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_2288a4f6.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_504d320d.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_5992faf7.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_5ef3e95c.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_986d6c22.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_9adc92c2.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_9cd1160a.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_a51bb26a.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_beebcee2.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_c04300ef.out.jpg
Processed and saved: gs://datalakes-ing3/1_staging/001_cb004eea.

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# Initialize Pinecone
api_key = os.getenv('PINECONE_API_KEY')
pinecone = Pinecone(api_key=api_key)
index = pinecone.Index('datalakes-ing3-curated')

NameError: name 'os' is not defined

In [8]:
from PIL import Image
import io
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output) # Load VGG16 excluding top layers

def extract_features(image_bytes):
    img = Image.open(io.BytesIO(image_bytes))
    img = img.resize((224, 224))  # Resize the image as required

    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    flat_feature = vgg16_feature.flatten()
    return flat_feature

In [11]:
from google.cloud import storage
import os

bucket_name = 'datalakes-ing3'
bucket = storage_client.bucket(bucket_name)

# List JPEG images in the bucket
blobs = storage_client.list_blobs(bucket_name, prefix='1_staging/', delimiter='/')

for blob in blobs:
    if blob.name.endswith('.jpg'):
        try:
            img_content = blob.download_as_bytes()
            features = extract_features(img_content)
            features /= np.linalg.norm(features)  # Normalize for cosine similarity to work
            index.upsert([(blob.name, features)])  # Index the feature vector in Pinecone

        except Exception as e:
            print(f"Error processing {blob.name}: {e}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [12]:
status = index.describe_index_stats()
print(status)

{'dimension': 4096,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1801}},
 'total_vector_count': 1801,
 'vector_type': 'dense'}


In [4]:
import os
from pinecone import Pinecone
pinecone = Pinecone(api_key="pcsk_6mfAsZ_7xBhAxC4ieaUqRR6S3josinEyvg37siV71awC2gXXPdk7Ycnda9h8HkB5ZAPp5N")
index = pinecone.Index('datalakes-ing3-curated')
index_stats = pinecone.describe_index("datalakes-ing3-curated")
print(index_stats)

{'deletion_protection': 'disabled',
 'dimension': 4096,
 'host': 'datalakes-ing3-curated-8v2ezu9.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'datalakes-ing3-curated',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}
